# Mnist with PyTorch

In [6]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torch.optim as optim

## Data Loader:

In [7]:
batch_size_test, batch_size_train = 64, 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) 

train_loader = torch.utils.data.DataLoader(datasets.MNIST('/files/', train=True, download=True,transform=transform),
                                           batch_size=batch_size_train, shuffle=True)

test_loader =  torch.utils.data.DataLoader(datasets.MNIST('/files/', train=False, download=True,transform=transform), 
                                           batch_size=batch_size_test, shuffle=True)

## Model:

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [14]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 100


network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

## Train:

In [18]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            
        train_losses.append(loss.item())
        train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        
        #torch.save(network.state_dict(), save_name+'_model.pth')
        #torch.save(optimizer.state_dict(), save_name+'_optimizer.pth')

## Test:

In [20]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print('Test set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [21]:
train_losses = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

Test set: Avg. Loss: 0.0970, Accuracy: 9694/10000 (97%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.342209
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.249683
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.137283
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.248723
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.390595
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.241265
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.172911
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.194119
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.324381
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.215002
Test set: Avg. Loss: 0.0868, Accuracy: 9732/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.107899
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.183279
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.265863
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.202454
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.308068
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.119186
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.266150
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.32280